In [1]:
%pip install pandas
%pip install requests
%pip install alpaca-py
%pip install pytz

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\School\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\School\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
     ------------------------------------ --- 81.9/90.6 kB 4.8 MB/s eta 0:00:01
     ---------------------------------------- 90.6/90.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/122.5 kB ? eta -:--:--
   ---------------------------------------- 122.5/122.5 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.6 kB ? eta -:--:--
   ---------------------------------------- 71.6/71.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/463.6 kB ? eta -:--:--
   ------------------------------------- -- 430.1/463.6 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 463.6/463.6 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.6/2.0 MB 17.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.4/2.0 MB 17.5 MB/s eta 0:00:01
   ------------


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\School\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\School\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data import StockHistoricalDataClient
from datetime import datetime
import pandas as pd

def get_historical_data():
    stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")
    formatted_request = StockBarsRequest(
        symbol_or_symbols=["NVDA"], # This could change right now it is a hardcoded value but in the future it could be dynamic
        start=datetime(2025, 4, 18), # This could change right now it is a hardcoded value but in the future it could be dynamic 
        end=datetime(2026,2,12), # This could change right now it is a hardcoded value but in the future it could be dynamic 
        timeframe=TimeFrame.Minute
    )
    response = stock_client.get_stock_bars(formatted_request)
    #print(response)
    return response

Stock_Data = get_historical_data()

mydf = Stock_Data.df

print(mydf.columns) # This is just to check what columns we have in the DataFrame it also is basically a test to make sure the above works
print(mydf.describe())

mydf = mydf.reset_index()
mydf = mydf.set_index("timestamp")

# This just makes it so the timezone is the same timezone as us though it could change depending on what we want to do in the future
mydf = mydf.tz_convert("US/Central")
mydf = mydf.between_time("08:30", "15:00")
mydf = mydf[mydf.index.dayofweek < 5]

market_open = pd.to_datetime("08:30").time()
market_close = pd.to_datetime("15:00").time()


full_timeline = pd.date_range(start=mydf.index.min(),end=mydf.index.max(), freq="1min", tz="US/Central")
full_timeline = full_timeline[(full_timeline.time >= market_open) & (full_timeline.time <= market_close) & (full_timeline.dayofweek < 5)]

mydf = mydf.reindex(full_timeline)

# This is basically the santization though unsure if this is how we want to handle it.
# Right now this uses the last available data point to fill in the missing values for open, high, low, and close, and fills in missing volume with 0.

# *********************** HAVE TO ASK *************************
# Do we want to handle the missing data differently? 

mydf[['open', 'high', 'low', 'close']] = mydf[['open', 'high', 'low', 'close']].ffill()
mydf['volume'] = mydf['volume'].fillna(0)


print(mydf.describe())

# ************************* SUMMARY *************************
# So all in all the DataFrame is called mydf and it has the following columns: open, high, low, close, volume. 
# It is in Central Time Zone though could change depending on what we want to do
# Also think this could all be converted to a function that takes in the symbol, start date, end date, and timezone as parameters and then returns the cleaned DataFrame. 
# Just not entirely sure what we want or how this is all going to be used in the future so I just left it as is for now.


Index(['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'], dtype='object')
                open           high            low          close  \
count  194786.000000  194786.000000  194786.000000  194786.000000   
mean      169.172977     169.241115     169.103207     169.173106   
std        24.338842      24.346925      24.331458      24.338446   
min        95.090000      95.140100      95.040000      95.080000   
25%       157.859900     157.909925     157.810000     157.858075   
50%       178.050000     178.130000     177.980000     178.050000   
75%       185.000000     185.060000     184.930000     185.000000   
max       211.985000     212.189900     211.740000     211.980000   

             volume    trade_count           vwap  
count  1.947860e+05  194786.000000  194786.000000  
mean   1.958244e+05    2221.902431     169.172825  
std    6.613354e+05    3870.025792      24.339137  
min    1.390000e+02       1.000000      95.087022  
25%    5.241000e+03     102.0